## Warehouse test

In [17]:
import os
import numpy as np
from environment import Environment
from openai import OpenAI
import copy
import configs
import pickle
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import ollama

In [18]:
with open('./test_set/{}_{}agents.pth'.format('warehouse', 32), 'rb') as f:
    tests = pickle.load(f)

In [19]:
# negotiator를 구성하기 위한 union find 알고리즘
def find(parent, i):
    if parent[i] == i:
        return i
    else:
        return find(parent, parent[i])

def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)

    if xroot != yroot:
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

def merge_sets(lists):
    element_to_index = {}
    for i, subset in enumerate(lists):
        for element in subset:
            element_to_index[element] = i

    parent = [i for i in range(len(lists))]
    rank = [0] * len(lists)

    for subset in lists:
        first_element = subset[0]
        for element in subset[1:]:
            union(parent, rank, find(parent, element_to_index[first_element]), find(parent, element_to_index[element]))

    new_sets = {}
    for element in element_to_index:
        root = find(parent, element_to_index[element])
        if root not in new_sets:
            new_sets[root] = set()
        new_sets[root].add(element)

    return [list(s) for s in new_sets.values()]

def find_direction(sentence):
    # 모든 단어를 찾기 위한 정규 표현식 패턴
    words = re.findall(r'"([^"]*)"|\b\w+\b', sentence)
    directions = ["north", "west", "south", "east"]
    # 모든 단어를 뒤에서부터 확인
    for word in reversed(words):
        if word.lower() in directions:
            return word
    return 'none'

def determine_direction(x, y, x_finish, y_finish):

    try:
        dx = int(x_finish) - x
        dy = int(y_finish) - y
        
        if dx > 0 and dy > 0:
            return "northeast"
        elif dx > 0 and dy < 0:
            return "southeast"
        elif dx < 0 and dy > 0:
            return "northwest"
        elif dx < 0 and dy < 0:
            return "southwest"
        elif dx > 0 and dy == 0:
            return "east"
        elif dx < 0 and dy == 0:
            return "west"
        elif dx == 0 and dy > 0:
            return "north"
        elif dx == 0 and dy < 0:
            return "south"
        else:
            return "Same location"
    except ValueError:
        return "error"
    

# 방향 정의
directiondict = {
    'stay': 0, 'north': 1, 'south': 2, 'west': 3, 'east': 4
}

In [20]:
# 프롬프트
class gpt4pathfinding:
    def navigate(self, agent, direction, east, west, north, south, last_action, coordinates, agent_FOV):
        response = ollama.generate(
        model="llama3",
        prompt=f"""
                <s>[INST] <<SYS>>
                당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다.
                <</SYS>>

                당신의 지난 방문한 좌표들은 다음과 같습니다.
                오른쪽으로 갈수록 당신이 최근에 방문한 좌표입니다.
                {coordinates}

                1. 여기서 만약 3번 이상 반복되는 좌표가 있다면, 그 좌표를 지정하고 dead end라는 답을 내세요. 아니라면, no라는 답을 내세요.

                당신의 목표 지점은 당신의 {direction} 방향에 있습니다.

                당신은 가로로 긴 창고 환경에 있기 때문에, y축으로 이동이 가능할 때는 y축을 먼저 이용하는 것이 강력하게 추천됩니다.
                {east}
                {west}
                {south}
                {north}

                당신은 한 번에 한 번만 움직일 수 있습니다.
                당신의 행동에는 동쪽, 서쪽, 남쪽, 북쪽, 그리고 가만히 있기 5가지의 선택지가 있습니다.
                
                당신은 다음 어느 방향으로 움직일지 결정하세요.

                아래는 답변의 예시들입니다.
                Be sure to answer in the format of the examples below.
                Don't add a description on your answer.

                EXAMPLE 1

                1. no
                2. north

                EXAMPLE 2

                1. dead end at (2, 3)
                2. south
                [/INST]
                """
        )
        return agent, response['response']
    
    def escape(self, deadenddirection, east, west, north, south):
        response = ollama.generate(
        model="llama3",
        prompt=f"""
                <s>[INST] <<SYS>>
                당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다.
                <</SYS>>

                당신은 dead end에서 똑같은 위치를 반복하고 있는 상태입니다.

                반복되는 좌표의 위치는 당신 기준 {deadenddirection}입니다.

                당신은 dead end로부터 최대한 멀리 떨어져야 합니다.

                당신의 행동에는 east, west, south, north의 선택지가 있습니다.

                {east}
                {west}
                {south}
                {north}

                1. 당신은 다음 어느 방향으로 움직일지 결정하세요.

                아래는 답변의 예시입니다. Answer with only one word.

                EXAMPLE

                south
                [/INST]
                """
        )
        return response['response']
    
    def give_way(self, east, west, north, south, direction):
        response = ollama.generate(
        model="llama3",
        prompt=f"""
                <s>[INST] <<SYS>>
                당신은 방향 감각이 뛰어난 탐험가입니다. 당신은 목표물에 도달하기 위해서 어떤 경로를 탐색하야 할지 잘 압니다.
                <</SYS>>

                You MUST avoid collisions with prior path finder.
                You can only move east, west, south and north or stay.
                So, your answer will be {{east}}, {{west}}, {{south}}, {{north}}, or {{stay}}.
                You can only make one move at a time.
                {east}
                {west}
                {south}
                {north}
                To the {direction} of you is prior path finder.
                I highly recommend going to the 90-degree turn rather than the other side of the pathfinder, but you can ignore this instruction depending on your situation.

                1. Which direction do you move next?

                아래는 답변의 예시입니다. Answer with only one word.

                EXAMPLE

                south
                """
        )
        return response['response']
    
pathfinder = gpt4pathfinding()

In [21]:
def run_one_example(example, max_steps):

  num_agents = len(example[1])
  map_width = len(example[0]) - 1

  env = Environment()
  env.load(np.array(example[0]), np.array(example[1]), np.array(example[2]))

  def parameter_navigate(i):

    observe = env.observe()

    a_obs = observe[0][i][1]
    x = observe[1][i][1]
    y = map_width-observe[1][i][0]
    x_finish = example[2][i][1]
    y_finish = map_width-example[2][i][0]
    direction = determine_direction(x, y, x_finish, y_finish)

    if a_obs[4][3] == 0:
      west = """To your west lies a path."""
    else:
      west = """Your west is blocked by a wall."""

    if a_obs[4][5] == 0:
      east = """To your east lies a path."""
    else:
      east = """Your east is blocked by a wall."""

    if a_obs[5][4] == 0:
      south = """To your south lies a path."""
    else:
      south = """Your south is blocked by a wall."""

    if a_obs[3][4] == 0:
      north = """To your north lies a path."""
    else:
      north = """Your north is blocked by a wall."""

    if len(action_list[i]) == 0:
      last_action = """"""
    elif action_list[i][-1] in ['West', 'West.']:
      last_action = f"""You came from east."""
    elif action_list[i][-1] in ['East', 'East.']:
      last_action = f"""You came from west."""
    elif action_list[i][-1] in ['South', 'South.']:
      last_action = f"""You came from north."""
    elif action_list[i][-1] in ['North', 'North.']:
      last_action = f"""You came from south."""
    elif action_list[i][-1] in ['Stay', 'Stay.']:
      last_action = f"""You paused once for another agent or you reached your goal."""
    else:
      last_action = """"""

    coordinates = f"""{coordinate_list[i]}"""

    agent_FOV = '\n'.join(' '.join(map(str, row)) for row in env.observe()[0][i][1][2:7, 2:7])

    return direction, east, west, north, south, last_action, coordinates, agent_FOV
  
  def parameter_give_way(i):

    observe = env.observe()

    a_obs = observe[0][i][1]
    x = observe[1][i][1]
    y = map_width-observe[1][i][0]
    x_finish = example[2][i][1]
    y_finish = map_width-example[2][i][0]

    if a_obs[4][3] == 0:
        west = """To your west lies a path."""
    else:
        west = """Your west is blocked by a wall."""

    if a_obs[4][5] == 0:
        east = """To your east lies a path."""
    else:
        east = """Your east is blocked by a wall."""

    if a_obs[5][4] == 0:
        south = """To your south lies a path."""
    else:
        south = """Your south is blocked by a wall."""

    if a_obs[3][4] == 0:
        north = """To your north lies a path."""
    else:
        north = """Your north is blocked by a wall."""

    # prior agent의 direction 구하기
    agent_map = env.observe()[0][i][0][2:7, 2:7]
    center = (2, 2)
    max_position = np.unravel_index(np.argmax(agent_map), agent_map.shape)

    row_diff = center[0] - max_position[0]
    col_diff = center[1] - max_position[1]

    if row_diff > 0 and col_diff > 0:
        direction = "northwest"
    elif row_diff > 0 and col_diff == 0:
        direction = "north"
    elif row_diff > 0 and col_diff < 0:
        direction = "northeast"
    elif row_diff == 0 and col_diff > 0:
        direction = "west"
    elif row_diff == 0 and col_diff < 0:
        direction = "east"
    elif row_diff < 0 and col_diff > 0:
        direction = "southwest"
    elif row_diff < 0 and col_diff == 0:
        direction = "south"
    elif row_diff < 0 and col_diff < 0:
        direction = "southeast"
    else:
        direction = "center"

    return east, west, north, south, direction

  
  #에이전트들의 지난번 액션들과 좌표
  action_list = [[] for _ in range(num_agents)]
  coordinate_list = [[] for _ in range(num_agents)]
  step_list = [0 for _ in range(num_agents)]
  steps = 0

  # turn
  while steps < max_steps:

    for i in range(num_agents):
      if not np.array_equal(env.agents_pos[i], env.goals_pos[i]):
        step_list[i] += 1

    for i in range(num_agents):
      x_coordinate = env.observe()[1][i][1]
      y_coordinate = map_width-env.observe()[1][i][0]

      if len(coordinate_list[i]) >= 8:
        coordinate_list[i].pop(0)

      coordinate_list[i].append([x_coordinate, y_coordinate])

    step = [0 for i in range(num_agents)]

    #각 에이전트들의 시야에 있는 자신과 다른 에이전트들
    FOV_agents = []
    for i in range(num_agents):
      if np.any(env.observe()[0][i][0][2:7, 2:7]):
        non_zero_elements = env.observe()[0][i][0][2:7, 2:7][env.observe()[0][i][0][2:7, 2:7] != 0].tolist()
        non_zero_elements = [element - 1 for element in non_zero_elements]
        non_zero_elements.append(i)
        FOV_agents.append(non_zero_elements)

    #알고리즘을 이용해 연결된 집합 찾기
    connected_sets = merge_sets(FOV_agents)

    #연결이 있는 모든 에이전트들
    deadlocked_agents = [item for sublist in connected_sets for item in sublist]

    parameters = []

    for i in range(num_agents):

      if np.array_equal(env.agents_pos[i], env.goals_pos[i]):
        step[i] = directiondict['stay']
        continue  # 다음 에이전트로 넘어감

      #교착상태에 빠지지 않은 에이전트는 독립적으로 navigate
      if i not in deadlocked_agents:

        direction, east, west, north, south, last_action, coordinates, agent_FOV = parameter_navigate(i)

        parameters.append([i, direction, east, west, north, south, last_action, coordinates, agent_FOV])

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
      # 각 프롬프트에 대해 비동기 작업 제출
      futures = [executor.submit(pathfinder.navigate, parameter[0], parameter[1], parameter[2], parameter[3], parameter[4], parameter[5], parameter[6], parameter[7], parameter[8]) for parameter in parameters]
      
      results = []

      # 각 작업의 결과 출력
      for future in concurrent.futures.as_completed(futures):
          result = future.result()
          results.append(result)

    for (i, result) in results: 
        lines = result.split('\n')

        # 첫 번째 줄과 두 번째 줄을 변수에 할당
        result1 = lines[0]
        result2 = lines[1]

        if "dead end" in result1:
          try:
            coords = result1.split(" at ")[1].strip("()").split(", ")
            x_finish = int(coords[0])
            y_finish = int(coords[1])
            x = env.observe()[1][i][1]
            y = map_width-env.observe()[1][i][0]
            deadenddirection = determine_direction(x, y, x_finish, y_finish)
            escape = pathfinder.escape(deadenddirection, east, west, north, south)
            direction = escape
          except (IndexError, ValueError):
            direction = 'stay'
        else:
          try:
            direction = result2.split(". ")[1]
          except IndexError:
            direction = 'stay'
        
        if direction not in directiondict:
          direction = 'stay'

        action_list[i].append(direction)

        step[i] = directiondict[direction]
      
    
    # 교착상태에 빠진 에이전트에 대해
    for connected_set in connected_sets:
      
      results = []

      parameters = []

      for i in connected_set: # 여기서 i는 agent의 번호들
      
      #각각의 에이전트에 대해 원래 계획된 경로를 계산

        direction, east, west, north, south, last_action, coordinates, agent_FOV = parameter_navigate(i)

        _, result = pathfinder.navigate(i, direction, east, west, north, south, last_action, coordinates, agent_FOV)

        lines = result.split('\n')

        # 첫 번째 줄과 두 번째 줄을 변수에 할당
        result1 = lines[0]
        result2 = lines[1]

        if "dead end" in result1:
          try:
            coords = result1.split(" at ")[1].strip("()").split(", ")
            x_finish = int(coords[0])
            y_finish = int(coords[1])
            x = env.observe()[1][i][1]
            y = map_width-env.observe()[1][i][0]
            deadenddirection = determine_direction(x, y, x_finish, y_finish)
            escape = pathfinder.escape(deadenddirection, east, west, north, south)
            direction = escape
          except (IndexError, ValueError):
            direction = 'stay'
        else:
          try:
            direction = result2.split(". ")[1]
          except IndexError:
            direction = 'stay'
        
        if direction not in directiondict:
          direction = 'stay'

        results.append(direction)

      env_copy = copy.deepcopy(env)
      planned_step = [0 for _ in range(num_agents)]
      for idx, agent_idx in enumerate(connected_set):
        planned_step[agent_idx] = directiondict[results[idx]]
      if -0.5 in env_copy.step(planned_step)[1]:
        collision = True
      else:
        collision = False

      if not collision:
        for idx, agent_idx in enumerate(connected_set):
          action_list[agent_idx].append(results[idx])
          step[agent_idx] = directiondict[results[idx]]
      # 충돌한다면 가장 큰 넘버의 에이전트가 우선권을 가지고, 나머지는 그를 피해가야 함 (이 부분을 좀더 정교화할 필요가 있음, 특히 3명 이상의 에이전트에 대한 경우. 일단 기본적으로는 얽힌 모든 에이전트가 가장 번호가 높은 에이전트에게서 도망가게.)
      # 특히, 먼저 도착한 에이전트는 장애물이 되어 그 멈춘 에이전트를 발견한 다른 에이전트의 heuristic map이 업데이트되어야 함 (멈춘 에이전트로 인해 예상치 못한 deadend가 생김)
      else:
      # 먼저 prior agent는 원래대로 이동하게 함
        prior_agent_number = max(connected_set)
        prior_agent_idx = connected_set.index(prior_agent_number)
        action_list[prior_agent_number].append(results[prior_agent_idx])
        step[prior_agent_number] = directiondict[results[prior_agent_idx]]
        for i in connected_set:
          if i != prior_agent_number:
              
              east, west, north, south, direction = parameter_give_way(i)

              result = pathfinder.give_way(east, west, north, south, direction)

              direction = result

              if direction not in directiondict:
                direction = 'stay'

              action_list[i] = []
      
              step[i] = directiondict[direction]

    observation, rewards, done, info = env.step(step)

    if done:
      break
    steps += 1

  return step_list

In [22]:
def simulate_all_tests(tests, max_steps):
    all_step_lists = []
    for example in tests:
        step_list = run_one_example(example, max_steps)
        all_step_lists.append(step_list)
        print(step_list)
    return all_step_lists

In [23]:
all_step_lists = simulate_all_tests(tests[0:100], 512)

[107, 151, 12, 26, 512, 84, 512, 88, 512, 154, 512, 168, 512, 512, 28, 100, 97, 79, 62, 512, 48, 512, 512, 143, 512, 512, 512, 512, 299, 512, 289, 44]
[123, 205, 105, 512, 53, 301, 512, 290, 135, 93, 97, 121, 456, 512, 90, 512, 56, 110, 77, 262, 59, 71, 35, 143, 8, 150, 125, 345, 134, 142, 509, 274]
[512, 512, 77, 21, 59, 55, 512, 170, 84, 42, 38, 133, 512, 512, 512, 159, 79, 106, 69, 34, 29, 140, 95, 168, 112, 512, 512, 512, 47, 139, 74, 179]
[512, 512, 40, 125, 100, 512, 93, 70, 46, 91, 424, 17, 512, 15, 80, 69, 125, 139, 93, 275, 173, 151, 512, 512, 59, 512, 147, 116, 26, 88, 21, 30]
[28, 512, 81, 48, 512, 31, 159, 68, 18, 157, 512, 58, 33, 178, 512, 126, 37, 189, 78, 58, 512, 26, 494, 128, 512, 33, 46, 11, 61, 512, 512, 512]
[96, 97, 127, 47, 169, 512, 142, 512, 512, 97, 512, 512, 512, 143, 512, 512, 284, 512, 512, 118, 512, 512, 30, 31, 44, 142, 22, 6, 82, 113, 256, 512]
[60, 240, 46, 259, 38, 366, 158, 57, 34, 512, 96, 79, 161, 9, 269, 56, 137, 512, 512, 78, 49, 64, 63, 172, 287,

IndexError: list index out of range

In [24]:
data = [[107, 151, 12, 26, 512, 84, 512, 88, 512, 154, 512, 168, 512, 512, 28, 100, 97, 79, 62, 512, 48, 512, 512, 143, 512, 512, 512, 512, 299, 512, 289, 44],
[123, 205, 105, 512, 53, 301, 512, 290, 135, 93, 97, 121, 456, 512, 90, 512, 56, 110, 77, 262, 59, 71, 35, 143, 8, 150, 125, 345, 134, 142, 509, 274],
[512, 512, 77, 21, 59, 55, 512, 170, 84, 42, 38, 133, 512, 512, 512, 159, 79, 106, 69, 34, 29, 140, 95, 168, 112, 512, 512, 512, 47, 139, 74, 179],
[512, 512, 40, 125, 100, 512, 93, 70, 46, 91, 424, 17, 512, 15, 80, 69, 125, 139, 93, 275, 173, 151, 512, 512, 59, 512, 147, 116, 26, 88, 21, 30],
[28, 512, 81, 48, 512, 31, 159, 68, 18, 157, 512, 58, 33, 178, 512, 126, 37, 189, 78, 58, 512, 26, 494, 128, 512, 33, 46, 11, 61, 512, 512, 512],
[96, 97, 127, 47, 169, 512, 142, 512, 512, 97, 512, 512, 512, 143, 512, 512, 284, 512, 512, 118, 512, 512, 30, 31, 44, 142, 22, 6, 82, 113, 256, 512],
[60, 240, 46, 259, 38, 366, 158, 57, 34, 512, 96, 79, 161, 9, 269, 56, 137, 512, 512, 78, 49, 64, 63, 172, 287, 512, 276, 76, 512, 175, 317, 109],
[512, 161, 45, 12, 512, 512, 85, 141, 142, 71, 512, 498, 512, 117, 57, 512, 81, 512, 512, 105, 19, 146, 19, 52, 276, 116, 5, 214, 351, 10, 512, 87],
[96, 121, 145, 131, 38, 84, 129, 40, 512, 491, 512, 48, 512, 512, 512, 27, 512, 162, 113, 65, 417, 13, 512, 512, 21, 96, 146, 77, 121, 58, 67, 256],
[120, 63, 94, 59, 321, 512, 244, 76, 165, 320, 79, 512, 137, 158, 108, 263, 139, 512, 130, 15, 331, 499, 102, 46, 512, 188, 103, 512, 163, 41, 204, 53],
[512, 512, 27, 127, 30, 512, 512, 512, 61, 131, 92, 80, 45, 58, 79, 35, 91, 171, 139, 512, 512, 29, 45, 512, 101, 512, 85, 510, 512, 96, 56, 61],
[512, 112, 512, 196, 77, 512, 76, 61, 142, 35, 512, 35, 121, 161, 512, 71, 130, 512, 46, 55, 64, 137, 107, 159, 45, 158, 492, 46, 157, 144, 116, 114],
[151, 113, 512, 512, 512, 512, 188, 512, 108, 153, 512, 499, 151, 512, 120, 104, 487, 66, 71, 133, 512, 82, 89, 25, 512, 512, 512, 512, 510, 160, 64, 146]]

In [25]:
data_array = np.array(data)

In [27]:
mean_values_including_512 = np.mean(data_array)
print(mean_values_including_512)

227.5096153846154


In [28]:
total_elements = data_array.size
count_512 = np.sum(data_array == 512)

In [29]:
print(count_512)

111


In [30]:
32*13

416